<a href="https://colab.research.google.com/github/gifted-clover95/photo-gallery-for-everyone/blob/main/no%2Be038.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class photo_gallery_for_everyone :
    def __init__(self, file_name):
        self.file_name = file_name
        self.count = 0
        self.height = 670
        self.width = 1280
        self.img = np.ones((self.height, self.width, 3), dtype=np.uint8) * 255
        self.origin = (self.height, self.width)
        self.scale = 1.0

    def set_origin_center(self):
        self.origin = (self.height//2, self.width//2)

    def show(self,im = 0):
        if type(im) != np.ndarray:
            im = self.img
        #plt.imshow(im)
        #plt.show()
        self.count += 1
        plt.imsave(self.file_name + str(self.count).zfill(3) + '.png', im)


In [3]:
photo_gallery = photo_gallery_for_everyone('no+e_photo_gallery_038_')

In [4]:
class lattice :
    def __init__(self,x,y,d):
        self.angle = np.pi / 3.0
        self.sin_angle = np.sin( self.angle )
        self.cos_angle = np.cos( self.angle )
        self.x = x
        self.y = y
        self.direction = d

    def turn_left(self):
        self.direction= (self.direction+1) % 6
        return

    def turn_right(self):
        self.direction= (self.direction-1) % 6
        return

    def move_forward(self):
        if self.direction == 0:
            self.x += 1
        elif self.direction == 1:
            self.y += 1
        elif self.direction == 2:
            self.x -= 1
            self.y += 1
        elif self.direction == 3:
            self.x -= 1
        elif self.direction == 4:
            self.y -= 1
        elif self.direction == 5:
            self.x += 1
            self.y -= 1
        else :
            print("move_forward : error")
        return (self.x, self.y)

    def get_current_point(self):
        return (self.x, self.y)

    def transform_point(self, p):
        x, y = p
        return (x + self.cos_angle * y, self.sin_angle * y)

In [5]:
class terdragon :
    def __init__(self, n,x=0,y=0,d=0):
        self.n = n
        self.lattice = lattice(x,y,d)
        self.string_rewriting_rule = {'F':'F+F-F'}
        self.path = [self.lattice.get_current_point()]
        self.generate_path()

    def generate_string(self):
        s = 'F'
        for i in range(self.n):
            s = s.translate(str.maketrans(self.string_rewriting_rule))
        return s

    def generate_path(self):
        for c in  self.generate_string():
            if c == 'F':
                self.path.append(self.lattice.move_forward())
            elif c == '+':
                self.lattice.turn_left()
                self.lattice.turn_left()
            elif c == '-':
                self.lattice.turn_right()
                self.lattice.turn_right()
            else:
                print("generate_path : error")

In [6]:
class color_iterator():
    def __init__(self, initial = 0):
        self.edge, self.r =  divmod (initial, 256)

    def get_color(self):
        if self.edge == 0:
            retv = (255, 255-self.r, self.r)
        elif self.edge == 1:
            retv = (255-self.r, self.r, 255)
        else:
            retv = (self.r, 255, 255-self.r)

        self.r += 1
        if self.r == 256:
            self.r = 1
            self.edge += 1
            if self.edge == 3:
                self.edge = 0

        return retv

In [7]:
def line (img, p1, p2, q1, q2, color=(0,0,0)):
    x1 = p1 + 0.5; x2 = q1 + 0.5
    y1 = p2 + 0.5; y2 = q2 + 0.5
    h = abs(q1-p1); w = abs(q2-p2)
    M = max(h,w)+1
    eps = 1.0/M
    for t in np.linspace(0,1, M ):
        x = int(t*x1+(1-t)*x2)
        y = int(t*y1+(1-t)*y2)
        img[x, y, 0] =  color[0]
        img[x, y, 1] =  color[1]
        img[x, y, 2] =  color[2]

In [8]:
n = 6
eps = 10
T1 = terdragon(n)
T2 = terdragon(n, 0,0,1)
T3 = terdragon(n, 0,0,2)
T4 = terdragon(n, 0,0,3)
T5 = terdragon(n, 0,0,4)
T6 = terdragon(n, 0,0,5)

nrow = len(T1.path)
P = np.zeros((6, nrow, 2))
for i in range(nrow):
    P[0, i, :] = T1.lattice.transform_point(T1.path[i])
    P[1, i, :] = T2.lattice.transform_point(T2.path[i])
    P[2, i, :] = T3.lattice.transform_point(T3.path[i])
    P[3, i, :] = T4.lattice.transform_point(T4.path[i])
    P[4, i, :] = T5.lattice.transform_point(T5.path[i])
    P[5, i, :] = T6.lattice.transform_point(T6.path[i])


h = P[:,:,0].max() - P[:,:,0].min()
w = P[:,:,1].max() - P[:,:,1].min()
R = min( (photo_gallery.height-eps) / h, (photo_gallery.width-eps)/ w)
P = R * P; h = R * h; w = R * w

x = 0.5 * (photo_gallery.height-h) - P[:,:,0].min()
y = 0.5 * (photo_gallery.width-w) - P[:,:,1].min()
P = P + np.array([x,y])

photo_gallery.img[:,:,:] = 255
for i in range(1,nrow):
    line(photo_gallery.img,
         round(P[0][i-1][0]), round(P[0][i-1][1]),
         round(P[0][i][0]),round(P[0][i][1]), (255,0,0))
    line(photo_gallery.img,
         round(P[1][i-1][0]), round(P[1][i-1][1]),
         round(P[1][i][0]),round(P[1][i][1]), (0,255,0))
    line(photo_gallery.img,
         round(P[2][i-1][0]), round(P[2][i-1][1]),
         round(P[2][i][0]),round(P[2][i][1]), (0,0,255))
    line(photo_gallery.img,
         round(P[3][i-1][0]), round(P[3][i-1][1]),
         round(P[3][i][0]),round(P[3][i][1]), (255,0,0))
    line(photo_gallery.img,
         round(P[4][i-1][0]), round(P[4][i-1][1]),
         round(P[4][i][0]),round(P[4][i][1]), (0,255,0))
    line(photo_gallery.img,
         round(P[5][i-1][0]), round(P[5][i-1][1]),
         round(P[5][i][0]),round(P[5][i][1]), (0,0,255))

photo_gallery.show()

In [9]:
for n in range(1,8):
    eps = 10
    T1 = terdragon(n)
    T2 = terdragon(n, 0,0,1)
    x,y = T1.path[-1]
    T3 = terdragon(n, x,y,2)

    nrow = len(T1.path)
    P = np.zeros((3, nrow, 2))
    for i in range(nrow):
        P[0, i, :] = T1.lattice.transform_point(T1.path[i])
        P[1, i, :] = T2.lattice.transform_point(T2.path[i])
        P[2, i, :] = T3.lattice.transform_point(T3.path[i])

    h = P[:,:,0].max() - P[:,:,0].min()
    w = P[:,:,1].max() - P[:,:,1].min()
    R = min( (photo_gallery.height-eps) / h, (photo_gallery.width-eps)/ w)
    P = R * P; h = R * h; w = R * w

    x = 0.5 * (photo_gallery.height-h) - P[:,:,0].min()
    y = 0.5 * (photo_gallery.width-w) - P[:,:,1].min()
    P = P + np.array([x,y])

    photo_gallery.img[:,:,:] = 0
    for i in range(1,nrow):
        line(photo_gallery.img,
             round(P[0][i-1][0]), round(P[0][i-1][1]),
             round(P[0][i][0]),round(P[0][i][1]), (0,255,255))
        line(photo_gallery.img,
             round(P[1][i-1][0]), round(P[1][i-1][1]),
             round(P[1][i][0]),round(P[1][i][1]), (255, 0, 255))
        line(photo_gallery.img,
             round(P[2][i-1][0]), round(P[2][i-1][1]),
             round(P[2][i][0]),round(P[2][i][1]), (255,255,0))
    photo_gallery.show()

In [10]:
def tiling (n = 1, m1 = 3, m2 = 6):
    x = 0; y = 0
    eps = 10
    T1 = terdragon(n, x, y, 0)
    T2 = terdragon(n, x, y,1)
    x,y = T1.path[-1]
    x1,y1 = T1.path[-1]
    T3 = terdragon(n, x,y,2)
    nrow = len(T1.path)
    P = np.zeros((3 * m1 * m2, nrow, 2))
    for i in range(nrow):
        P[0, i, :] = T1.lattice.transform_point(T1.path[i])
        P[1, i, :] = T2.lattice.transform_point(T2.path[i])
        P[2, i, :] = T3.lattice.transform_point(T3.path[i])

    x,y = T3.path[-1]
    for k in range(1,m2):
        T1 = terdragon(n, x, y, 0)
        T2 = terdragon(n, x, y,1)
        x,y = T1.path[-1]
        T3 = terdragon(n, x,y,2)
        x,y = T3.path[-1]
        for i in range(nrow):
            P[3*k+0, i, :] = T1.lattice.transform_point(T1.path[i])
            P[3*k+1, i, :] = T2.lattice.transform_point(T2.path[i])
            P[3*k+2, i, :] = T3.lattice.transform_point(T3.path[i])

    for l in range(1,m1):
        x = x1; y = y1
        T1 = terdragon(n, x, y, 0)
        T2 = terdragon(n, x, y,1)
        x,y = T1.path[-1]
        x1,y1 = T1.path[-1]
        T3 = terdragon(n, x,y,2)
        for i in range(nrow):
            P[3*l*m2+0, i, :] = T1.lattice.transform_point(T1.path[i])
            P[3*l*m2+1, i, :] = T2.lattice.transform_point(T2.path[i])
            P[3*l*m2+2, i, :] = T3.lattice.transform_point(T3.path[i])
        x,y = T3.path[-1]
        for k in range(1,m2):
            T1 = terdragon(n, x, y, 0)
            T2 = terdragon(n, x, y,1)
            x,y = T1.path[-1]
            T3 = terdragon(n, x,y,2)
            x,y = T3.path[-1]
            for i in range(nrow):
                P[3*l*m2+3*k+0, i, :] = T1.lattice.transform_point(T1.path[i])
                P[3*l*m2+3*k+1, i, :] = T2.lattice.transform_point(T2.path[i])
                P[3*l*m2+3*k+2, i, :] = T3.lattice.transform_point(T3.path[i])

    h = P[:,:,0].max() - P[:,:,0].min()
    w = P[:,:,1].max() - P[:,:,1].min()
    R = min( (photo_gallery.height-eps) / h, (photo_gallery.width-eps)/ w)
    P = R * P; h = R * h; w = R * w

    x = 0.5 * (photo_gallery.height-h) - P[:,:,0].min()
    y = 0.5 * (photo_gallery.width-w) - P[:,:,1].min()
    P = P + np.array([x,y])

    photo_gallery.img[:,:,:] = 0
    for i in range(1,nrow):
        for k in range(m1*m2):
            line(photo_gallery.img,
                 round(P[3*k+0][i-1][0]), round(P[3*k+0][i-1][1]),
                 round(P[3*k+0][i][0]),round(P[3*k+0][i][1]), (0,255,255))
            line(photo_gallery.img,
                 round(P[3*k+1][i-1][0]), round(P[3*k+1][i-1][1]),
                 round(P[3*k+1][i][0]),round(P[3*k+1][i][1]), (255, 0, 255))
            line(photo_gallery.img,
                 round(P[3*k+2][i-1][0]), round(P[3*k+2][i-1][1]),
                 round(P[3*k+2][i][0]),round(P[3*k+2][i][1]), (255,255,0))
    photo_gallery.show()

In [11]:
tiling(1, 9,10)

In [12]:
tiling(2, 4,8)

In [13]:
tiling(3, 6,5)

In [14]:
tiling(4, 4,3)

In [15]:
tiling(7, 3,4)